# Week 4: The geometry of meaning

We're going to explore some basic forms of text analysis, using David Robinson's dataset of tweets made from the account of Donald J. Trump, as well as a dataset of nineteenth-century poetry and fiction, which is divided by date, by genre, and also by reception (whether or not the volume got reviewed in an 'elite' journal).

To begin, let's import some modules we're going to need later, and also read in the Trump data.

In [1]:
import os, csv, math
import pandas as pd
import numpy as np

from collections import Counter
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer

cwd = os.getcwd()
print('Current working directory: ' + cwd + '\n')
      
relativepath = os.path.join('..', 'data', 'weekfour', 'trump.csv')
trump = pd.read_csv(relativepath)

Current working directory: /Users/tunder/Dropbox/courses/2017datasci/04-Geometry



## Different ways of identifying "distinctive" words

In this section we'll explore Dunning's log-likelihood, and also think about the strengths and weaknesses of "distinctive" words as evidence.

First let's glance at the Trump dataset.

In [2]:
trump.head()

,Unnamed: 0,Unnamed: 0.1,text,favorited,favoriteCount,replyToSN,created,truncated,replyToSID,id,replyToUID,statusSource,screenName,retweetCount,isRetweet,retweeted,longitude,latitude
0,0,1,My economic policy speech will be carried live...,False,9214,NaN,2016-08-08 15:20:44,False,NaN,762669882571980801,NaN,"<a href=""http://twitter.com/download/android"" ...",realDonaldTrump,3107,False,False,NaN,NaN
1,1,2,"Join me in Fayetteville, North Carolina tomorr...",False,6981,NaN,2016-08-08 13:28:20,False,NaN,762641595439190016,NaN,"<a href=""http://twitter.com/download/iphone"" r...",realDonaldTrump,2390,False,False,NaN,NaN
2,2,3,"#ICYMI: ""Will Media Apologize to Trump?"" https...",False,15724,NaN,2016-08-08 00:05:54,False,NaN,762439658911338496,NaN,"<a href=""http://twitter.com/download/iphone"" r...",realDonaldTrump,6691,False,False,NaN,NaN
3,3,4,"Michael Morell, the lightweight former Acting ...",False,19837,NaN,2016-08-07 23:09:08,False,NaN,762425371874557952,NaN,"<a href=""http://twitter.com/download/android"" ...",realDonaldTrump,6402,False,False,NaN,NaN
4,4,5,The media is going crazy. They totally distort...,False,34051,NaN,2016-08-07 21:31:46,False,NaN,762400869858115588,NaN,"<a href=""http://twitter.com/download/android"" ...",realDonaldTrump,11717,False,False,NaN,NaN


#### Basic functions

For a lot of the work we do today, we're going to want to construct dictionaries that hold the frequencies of words in different categories: poetry or fiction, Trump-iphone or Trump-android. To do this we'll need to break text into words, count the words in each text, and then add up the counts by category.

Let's define some functions that do this. (You can find more polished versions of these functions in the ```nltk``` module.)


In [3]:
def tokenize(astring):
    ''' Breaks a string into words, and counts them.
    Designed so it strips punctuation and lowercases everything,
    but doesn't separate hashtags and at-signs.
    '''
    wordcounts = Counter()
    # create a counter to hold the counts
    
    tokens = astring.split()
    for t in tokens:
        word = t.strip(',.!?:;-—()<>[]/"\'').lower()
        wordcounts[word] += 1
        
    return wordcounts

def addcounters(counter2add, countersum):
    ''' Adds all the counts in counter2add to countersum.
    Because Counters(like dictionaries) are mutable, it
    doesn't need to return anything.
    '''
    
    for key, value in counter2add.items():
        countersum[key] += value

def create_vocab(seq_of_strings, n):
    ''' Given a sequence of text snippets, this function
    returns the n most common words. We'll use this to
    create a limited 'vocabulary'.
    '''
    vocab = Counter()
    for astring in seq_of_strings:
        counts = tokenize(astring)
        addcounters(counts, vocab)
    topn = [x[0] for x in vocab.most_common(n)]
    return topn

# Let's test the vocabulary function.
vocab = create_vocab(trump['text'], 4000)
vocab[0:10]
        

['the', 'to', 'and', 'a', 'in', 'is', 'i', 'you', 'of', 'will']

#### A few more basic functions

Once we have a vocabulary, we're going to want to divide our texts into categories, create Counters summing the word frequencies in those categories, and then compare the two Counters to find words that are overrepresented in one category relative to the other.

There are several ways we could define "overrepresented." We'll use Robinson's simple log-odds measure, as well as Dunning's log-likelihood.

In [4]:
def logodds(countsA, countsB, word):
    ''' Straightforward.
    '''
    
    odds = (countsA[word] + 1) / (countsB[word] + 1)
    
    # Why do we add 1 on both sides? Two reasons. The hacky one is 
    # that otherwise we'll get a division-by-zero error whenever
    # word isn't present in countsB. The more principled reason
    # is that this technique (called Laplacian smoothing) tends
    # to reduce the dramatic disproportion likely to be found in
    # very rare words.
    
    return math.log(odds)

def signed_dunnings(countsA, totalA, countsB, totalB, word):
    ''' Less straightforward. This function calculates a signed (+1 / -1)
    version of Dunning's log likelihood. Intuitively, this is a number 
    that gets larger as the frequency of the word in our two corpora
    diverges from its EXPECTED frequency -- i.e., the frequency it would
    have if it were equally distributed over both. But it also tends to get
    larger as the raw frequency of the word increases.
    
    Note that this function requires two additional arguments:
    the total number of words in A and B. We could calculate that inside
    the function, but it's faster to calculate it just once, outside the function.
    
    Also note: the strict definition of Dunnings has no 'sign': it gets bigger
    whether a word is overrepresented in A or B. I've edited that so that Dunnings
    is positive if overrepresented in A, and negative if overrepresented in B.
    '''
    if word not in countsA and word not in countsB:
        return 0
    
    # the raw frequencies of this word in our two corpora
    # still doing a little Laplacian smoothing here
    a = countsA[word] + 0.1
    b = countsB[word] + 0.1
    
    # now let's calculate the expected number of times this
    # word would occur in both if the frequency were constant
    # across both
    overallfreq = (a + b) / (totalA + totalB)
    expectedA = totalA * overallfreq
    expectedB = totalB * overallfreq
    
    # and now the Dunning's formula
    dunning = 2 * ((a * math.log(a / expectedA)) + (b * math.log(b / expectedB)))
    
    if a < expectedA:
        return -dunning
    else:   
        return dunning

# a list of common words is often useful
stopwords = {'a', 'an', 'are', 'and', 'but', 'or', 'that', 'this', 'so', 
             'all', 'at', 'if', 'in', 'i', 'is', 'was', 'by', 'of', 'to', 
             'the', 'be', 'you', 'were'}

# finally, one more function: given a list of tuples like
testlist = [(10, 'ten'), (2000, 'two thousand'), (0, 'zero'), (-1, 'neg one'), (8, 'eight')]
# we're going to want to sort them and print the top n and bottom n

def headandtail(tuplelist, n):
    tuplelist.sort(reverse = True)
    print("TOP VALUES:")
    for i in range(n):
        print(tuplelist[i][1], tuplelist[i][0])
    
    print()
    print("BOTTOM VALUES:")
    lastindex = len(tuplelist) - 1
    for i in range(lastindex, lastindex - n, -1):
        print(tuplelist[i][1], tuplelist[i][0])
        
headandtail(testlist, 2)
    

TOP VALUES:
two thousand 2000
ten 10

BOTTOM VALUES:
neg one -1
zero 0


## Exercise 1: Is Dunning's a better measure than logodds for Trump's tweets?

Let's put all these functions together to answer that question.

I've sketched the outline of a program below in "pseudocode," which
describes what needs to be done. Translate that into real Python code, using
the functions defined above. First use Robinson's logodds function and try to
replicate his results. See what happens if you do (or don't) remove stopwords
and tweets that begin with a quote.
                                                   
Then edit your code to use Dunning's log likelihood. Does that seem to be a better (more revealing) measure of overrepresentation? How would we decide?

In [8]:
vocab = create_vocab(trump['text'], 5000)
vocab = list(set(vocab) - stopwords)

numrows = trump.shape[0]
android = Counter()
iphone = Counter()

#skipped = 0
for i in range(numrows):
    # if trump['text'][i].startswith('"'):
        #skipped += 1
        # continue
        
    counts = tokenize(trump['text'][i])
    if 'iphone' in trump['statusSource'][i]:
        addcounters(counts, iphone)
    elif 'android' in trump['statusSource'][i]:
        addcounters(counts, android)
#print(skipped)
iphonesum = sum(iphone.values())
androidsum = sum(android.values())

tuplelist = []

for word in vocab:
    g = signed_dunnings(iphone, iphonesum, android, androidsum, word)
    # g = logodds(iphone, android, word)
    tuplelist.append((g, word))

headandtail(tuplelist, 25)

TOP VALUES:
#trump2016 273.5841360055747
thank 196.81395689400256
#makeamericagreatagain 120.28324219640973
join 72.69759231558339
#americafirst 48.15900365424501
#imwithyou 40.72854875396748
#votetrump 40.16398066603733
tomorrow 33.61963686849273
#crookedhillary 28.299637642368594
#trumppence16 28.29096593144873
soon 27.62623138606114
&amp 26.87770273072912
tickets 22.455508011339962
video 22.08448072761334
7pm 22.08448072761334
#rncincle 20.018560530293765
#inprimary 20.018560530293765
#maga 19.283927014638802
new 17.79952270616385
indiana 17.730138764444213
pennsylvania 16.154464660809687
#wiprimary 15.893025142878843
#fitn 15.893025142878843
safe 15.150939231055148
officers 13.83449313808438

BOTTOM VALUES:
@realdonaldtrump -96.7586196380446
@megynkelly -16.56740161733122
more -15.782782083381733
because -13.567850903933373
job -13.396043113762923
win -13.31962104674283
trump -12.813499152342672
big -11.93248427605737
a.m -11.28010329618746
very -11.060232026337701
badly -10.425110

## Exercise 2: Apply the same methods to a more literary dataset.

I've also provided a dataset of roughly 1026 snippets from nineteenth-century poetry and fiction. The code below should read it in. Run that, then copy and paste the code you worked up for Trump, and edit it so it provides the most distinctive words for poetry (versus fiction).

If we have time, it may also be worth distinguishing poetry reviewed in elite journals from poetry that wasn't.


In [25]:
relativepath = os.path.join('..', 'data', 'weekfour', 'poefic.csv')
poefic = pd.read_csv(relativepath)
poefic.head()

,date,author,title,genre,reception,text
0,1908,"Robins, Elizabeth,",The convert,fiction,elite,"looked like decent artisans, but more who bore..."
1,1871,"Lytton, Edward Bulwer Lytton,",The coming race,fiction,elite,"called the "" Easy Time "" (with which what I ma..."
2,1872,"Butler, Samuel,","Erewhon, or, Over the range",fiction,elite,the curtain ; on this I let it drop and retrea...
3,1900,"Barrie, J. M.",Tommy and Grizel,fiction,elite,"at you !"" he said. ""Dear eyes, "" said she. ""Th..."
4,1873,"Ritchie, Anne Thackeray,",Old Kensington,fiction,elite,"furious; I have not dared tell her, poor creat..."


In [1]:
# This code deleted, because it's now a homework problem.

## Using corpora to create a "meaning space."

Contrasting two corpora can be revealing, but sometimes we want to think about the relations between individual texts or words. To do that, we often represent them as vectors in a multi-dimensional space.

The simplest way to do this is to create a DataFrame where rows are documents and columns are word — a document-term matrix. Here's a function that does that. It requires a pre-defined vocabulary (list of words) as well as a list (or numpy vector) of texts.


In [31]:
def doc_term_matrix(vocab, textvector):
    ''' Transform the textvector into a document-term matrix
    with one column for each word in vocab.
    '''
    
    n = len(textvector)
    vocabset = set(vocab)
    # making a set so we can check membership quickly;
    # it's much faster in a set than in a list
    
    termdictionary = dict()
    for word in vocab:
        termdictionary[word] = np.zeros(n)
    for i, text in enumerate(textvector):
        counts = tokenize(text)
        for word, count in counts.items():
            if word in vocabset:
                termdictionary[word][i] += count
    
    dtmatrix = pd.DataFrame(termdictionary, columns = vocab)
    return dtmatrix

# A nice arcane trick to perform on a document-term matrix
# is to squash it into a smaller number of dimensions. This
# often reveals relationships between words that don't
# necessarily, literally occur together. The technique is called
# Latent Semantic Analysis.

def lsa_matrix(dtmatrix, vocab, number_of_dimensions):
    lsa = TruncatedSVD(number_of_dimensions, algorithm = 'arpack')
    dtm_lsa = lsa.fit_transform(dtmatrix)
    dtm_lsa = Normalizer(copy=False).fit_transform(dtm_lsa)
    lsamatrix = pd.DataFrame(lsa.components_, columns = vocab)
    
    return lsamatrix

def cosine_similarity(vector1, vector2):
    dot_product = np.sum(vector1 * vector2)
    # we assume these are numpy vectors and can be
    # multiplied elementwise
    
    magnitude = math.sqrt(sum([val**2 for val in vector1])) * math.sqrt(sum([val**2 for val in vector2]))
    if not magnitude:
        return 0
    else:
        return dot_product/magnitude

In [49]:
print(iphonesum, androidsum)

8531 15662


In [34]:
vocab = create_vocab(poefic['text'], 2500)
dtm = doctermmatrix(vocab, poefic['text'])
lsa = make_lsa_matrix(dtm, vocab, 25)

def find_match(amatrix, vocab, user_word):
    vocabset = set(vocab)
    numcol = amatrix.shape[1]
    user_vector = amatrix[user_word]
    tuplelist = []
    for i in range(numcol):
        cossim = cosine_similarity(amatrix.iloc[ : , i], user_vector)
        tuplelist.append((cossim, vocab[i]))
    headandtail(tuplelist, 10)
    

In [39]:
user_word = input('word? ')
find_match(lsa, vocab, user_word) 

word? herself
TOP VALUES:
herself 1.0000000000000002
she 0.7716452092647031
daughters 0.5875505691208556
impossible 0.5768217529798366
possible 0.5702868892412817
slight 0.5455494133385587
drawing 0.5351924949952669
daughter 0.5234257917168922
husband 0.5221905605041337
rooms 0.521308013989169

BOTTOM VALUES:
tower -0.5465893529018647
lights -0.5118834629986829
sounds -0.44183523251151824
forest -0.41912245104573653
hills -0.41777580862271546
hill -0.4089810930096797
echoes -0.4054595882372723
swift -0.4050970804953546
bark -0.3980934359379713
streams -0.39759991597819794
